In [2]:
from pathlib import Path
import sys

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

#sys.path.append('/eos/user/m/morwat/.local/lib/python3.11/site-packages/')

#for x in ['xtrack','xobjects','xdeps','xpart']:
   # sys.path.append(f'/eos/home-m/morwat/xsuite/{x}/')

In [3]:
from aper_package.interactive_tool import InteractiveTool

In [4]:
tool = InteractiveTool()

In [5]:
tool.show()